In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to extract property details
def extract_property_details(soup):
    properties = []
    
    # Find all property listings
    listings = soup.find_all('div', class_='property-listing-grid')
    
    for listing in listings:
        try:
            # Extract price
            price = listing.find('div', class_='pl-price').find('h3').text.strip()
        except AttributeError:
            price = 'N/A'
        
        try:
            # Extract title
            title = listing.find('div', class_='pl-title').text.strip()
        except AttributeError:
            title = 'N/A'

        try:
            # Extract property ID
            pid = listing.find('p').text.strip().replace("PID :", "")
        except AttributeError:
            pid = 'N/A'

        try:
            # Extract number of beds and baths
            details = listing.find('h6').text.strip()
        except AttributeError:
            details = 'N/A'
        
        properties.append({
            'Title': title,
            'Price': price,
            'PID': pid,
            'Details': details
        })
        
    return properties

# URL of the page to scrape
url = 'https://www.propertypro.ng/property-for-sale'

# Send a GET request to fetch the HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Page fetched successfully!")
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract property details
    properties = extract_property_details(soup)
    
    # Convert the data into a DataFrame
    df = pd.DataFrame(properties)
    
    # Save the data into a CSV file
    df.to_csv('propertypro_listings.csv', index=False)
    print("Data saved to 'propertypro_listings.csv'")
else:
    print(f"Failed to fetch page, status code: {response.status_code}")

Page fetched successfully!
Data saved to 'propertypro_listings.csv'


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Function to extract property details
def extract_property_details(soup):
    properties = []
    
    # Find all property listings
    listings = soup.find_all('div', class_='property-listing-grid')
    
    for listing in listings:
        try:
            # Extract price
            price = listing.find('div', class_='pl-price').find('h3').text.strip()
        except AttributeError:
            price = 'N/A'
        
        try:
            # Extract title
            title = listing.find('div', class_='pl-title').text.strip()
        except AttributeError:
            title = 'N/A'

        try:
            # Extract property ID
            pid = listing.find('p').text.strip().replace("PID :", "")
        except AttributeError:
            pid = 'N/A'

        try:
            # Extract number of beds and baths
            details = listing.find('h6').text.strip()
        except AttributeError:
            details = 'N/A'
        
        properties.append({
            'Title': title,
            'Price': price,
            'PID': pid,
            'Details': details
        })
        
    return properties

# URL template (with page number placeholder)
url_template = 'https://www.propertypro.ng/property-for-sale?page={}'

# Number of pages to crawl
num_pages = 10  # Adjust this based on how many pages you want to scrape

# List to store all scraped properties
all_properties = []

# Crawl through multiple pages
for page_num in range(1, num_pages + 1):
    print(f"Scraping page {page_num}...")
    # Generate the URL for the current page
    url = url_template.format(page_num)
    
    # Send a GET request to fetch the HTML content
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract property details from the page
        properties = extract_property_details(soup)
        
        # Add the scraped data to the list
        all_properties.extend(properties)
        
        # Delay to avoid overloading the server
        time.sleep(2)  # Sleep for 2 seconds between requests
    else:
        print(f"Failed to fetch page {page_num}, status code: {response.status_code}")

# Convert the data into a DataFrame
df = pd.DataFrame(all_properties)

# Save the data into a CSV file
df.to_csv('propertypro_listings_all_pages.csv', index=False)
print("Data saved to 'propertypro_listings_all_pages.csv'")

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Data saved to 'propertypro_listings_all_pages.csv'


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Function to extract property details for rent listings
def extract_rental_details(soup):
    properties = []
    
    # Find all property listings for rent
    listings = soup.find_all('div', class_='property-listing-grid')
    
    for listing in listings:
        try:
            # Extract price
            price = listing.find('div', class_='pl-price').find('h3').text.strip()
        except AttributeError:
            price = 'N/A'
        
        try:
            # Extract title
            title = listing.find('div', class_='pl-title').text.strip()
        except AttributeError:
            title = 'N/A'

        try:
            # Extract property ID
            pid = listing.find('p').text.strip().replace("PID :", "")
        except AttributeError:
            pid = 'N/A'

        try:
            # Extract number of beds and baths
            details = listing.find('h6').text.strip()
        except AttributeError:
            details = 'N/A'
        
        properties.append({
            'Title': title,
            'Price': price,
            'PID': pid,
            'Details': details
        })
        
    return properties

# URL template for houses for rent (with page number placeholder)
url_template = 'https://www.propertypro.ng/property-for-rent?page={}'

# Number of pages to crawl (you can adjust this)
num_pages = 10

# List to store all scraped properties
all_rental_properties = []

# Crawl through multiple pages for rent listings
for page_num in range(1, num_pages + 1):
    print(f"Scraping page {page_num} for rent listings...")
    url = url_template.format(page_num)
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        rental_properties = extract_rental_details(soup)
        all_rental_properties.extend(rental_properties)
        time.sleep(2)
    else:
        print(f"Failed to fetch page {page_num}, status code: {response.status_code}")

# Convert the data into a DataFrame
df_rentals = pd.DataFrame(all_rental_properties)

# Save the data into a CSV file
df_rentals.to_csv('propertypro_rent_listings.csv', index=False)
print("Data saved to 'propertypro_rent_listings.csv'")

Scraping page 1 for rent listings...
Scraping page 2 for rent listings...
Scraping page 3 for rent listings...
Scraping page 4 for rent listings...
Scraping page 5 for rent listings...
Scraping page 6 for rent listings...
Scraping page 7 for rent listings...
Scraping page 8 for rent listings...
Scraping page 9 for rent listings...
Scraping page 10 for rent listings...
Data saved to 'propertypro_rent_listings.csv'
